<a href="https://colab.research.google.com/github/Yasaman-habibi/Analysis-of-Sustainability-Reports/blob/main/FinalStep_02_FilteredData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files, drive
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import re

In [ ]:
drive.mount('/content/drive')

# 1. آپلود فایل‌های متنی
uploaded_Texts = files.upload()

Combined_path = "/content/drive/MyDrive/Combined_Texts"
os.makedirs(Combined_path, exist_ok=True)
Combined_file = os.path.join(Combined_path, "Combined_Texts.txt")

In [ ]:
# 2. آپلود فایل دیکشنری Loughran-McDonald

uploaded_Dic = files.upload()
for filename in uploaded_Dic.keys():
    df_dict = pd.read_excel(io.BytesIO(uploaded_Dic[filename]))

Dic_path = "/content/drive/MyDrive/sustainability_table"
os.makedirs(Dic_path, exist_ok=True)
Dic_file = os.path.join(Dic_path, "Loughran-McDonald.xlsx")

df_dict.to_excel(Dic_file, index=False)

In [4]:
#برای خوندن فایل اکسل تا نیاز به هر بار اپلود نباشد

df_dict = pd.read_excel("/content/drive/MyDrive/sustainability_table/Loughran-McDonald.xlsx")
keywords = pd.read_excel("/content/drive/MyDrive/sustainability_table/keywords.xlsx")

In [9]:
# 3. ساخت فایل keywords

keywords_path = "/content/drive/MyDrive/sustainability_table"
os.makedirs(keywords_path, exist_ok=True)
keywords_file = os.path.join(keywords_path, "keywords.xlsx")

# لیست کلمات کلیدی نهایی
default_keywords = [
     "Sustainable" , "Sustainability" ,
     "Development" , "Warming" , "Resource" ,
     "Environmental" ,  "governance" , "environment",
     "climate" , "Recycling" , "Energy",
     "Economic" , "Economy" , "Green",
     "Social" , "Society" , "materials" , "Ecosystem"
     "Carbon", "footprint" , "Pollutants " ,
     "Natural" , "Earth", "Air", "biodiversity",  "Crisis" ,
      "Atmospheric" , "Water" , "pollution" ,
     "Drought" , "Famine" , "Decarbonization" ,
     "Species" , "Demographic" , "Global" ,
      "Waste" , "agriculture" , "innovation",
     ]

updated_keywords_df = pd.DataFrame(sorted(set(default_keywords)), columns=["keyword"])
updated_keywords_df.to_excel(keywords_file, index=False)
keywords = updated_keywords_df["keyword"].str.lower().tolist()

In [ ]:
# 4. ساخت مجموعه واژه‌های پایداری از دیکشنری بر مبنای کلیدواژه نهایی

keywords_set = set(keywords)

sustain_terms = set()
for word in df_dict["Word"]:
    word_lower = str(word).strip().lower()
    if word_lower in keywords_set:
        sustain_terms.add(word_lower)

print(f"{len(sustain_terms)} sustainability-related terms extracted.")

sustain_Dic = df_dict[df_dict["Word"].str.lower().isin(sustain_terms)]

sustain_Dic_path = "/content/drive/MyDrive/sustainability_table"
os.makedirs(sustain_Dic_path, exist_ok=True)
sustain_Dic_file = os.path.join(sustain_Dic_path, "sustain_Dic.xlsx")

sustain_Dic.to_excel(sustain_Dic_file, index=False)

print(f"Filtered Excel file saved to: {sustain_Dic_path}")

In [12]:
# 5. خواندن فایل های متنی برای انجام process

text = ""
for filename in uploaded_Texts.keys():
    with open(filename, "r", encoding="utf-8") as file:
        text += file.read() + "\n\n"

def split_paragraphs(text):
    paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
    return paragraphs

paragraphs = split_paragraphs(text)

In [ ]:
# 6. فیلتر کردن متن با استفاده از دیکشنری ساخته شده بر اساس کلمات پایداری

sustain_Dic_path = "/content/drive/MyDrive/sustainability_table/sustain_Dic.xlsx"
sustain_Dic = pd.read_excel(sustain_Dic_path)

sustain_terms = set(sustain_Dic["Word"].dropna().str.lower())


#  تابع بررسی وجود حداقل یک واژه پایداری در هر پاراگراف
# def contains_sustain_terms(paragraph):
#     words = paragraph.lower().split()
#     words_clean = [word.strip(".,;:!?()[]{}\"'") for word in words]
#     return any(word in sustain_terms for word in words_clean)

#فرایند برای یک کلمه یا چند کلمه بعنوان یک عبارت در لیست کلیدواژه
sustain_terms_raw = sustain_Dic["Word"].dropna().str.strip().str.lower().tolist()
single_terms = set(term for term in sustain_terms_raw if len(term.split()) == 1)
multi_terms = set(term for term in sustain_terms_raw if len(term.split()) > 1)

def contains_sustain_terms(paragraph):
    para_lower = paragraph.lower()

    #  عبارات چندکلمه‌ای
    padded_para = f" {para_lower} "
    match_multi = any(f" {term} " in padded_para for term in multi_terms)

    #  کلمات تکی
    words_in_para = re.findall(r'\b\w+\b', para_lower)
    match_single = any(word in single_terms for word in words_in_para)

    return match_multi or match_single

# فیلتر کردن پاراگراف‌های مرتبط با پایداری
filtered_paragraphs = [para for para in paragraphs if contains_sustain_terms(para)]

print(f"{len(filtered_paragraphs)} paragraphs contain sustainability-related terms.")

In [ ]:
# 7. ترکیب فقط پاراگراف‌های مرتبط با پایداری

Combined_Sustain_text_path = os.path.join(Combined_path, "Combined_Sustain_text.txt")

# بررسی شماره فایل خروجی
def get_next_index(base_path, prefix="Combined_Sustain_text_", suffix=".txt"):
    existing_files = os.listdir(base_path)
    indices = []

    for fname in existing_files:
        match = re.match(fr"{re.escape(prefix)}(\d+){re.escape(suffix)}", fname)
        if match:
            indices.append(int(match.group(1)))
    return max(indices, default=0) + 1


file_index = get_next_index(Combined_path)
output_filename = f"Combined_Sustain_text_{file_index}.txt"
output_path = os.path.join(Combined_path, output_filename)

with open(output_path, "w", encoding="utf-8") as outfile:
    for filename in uploaded_Texts.keys():
        with open(filename, "r", encoding="utf-8", errors="ignore") as infile:
            content = infile.read()
            paragraphs = split_paragraphs(content)
            filtered_paragraphs = [para for para in paragraphs if contains_sustain_terms(para)]

            if filtered_paragraphs:
                outfile.write(f"===== Start of File: {filename} =====\n")
                for para in filtered_paragraphs:
                    outfile.write(para + "\n\n")
                outfile.write(f"===== End of File: {filename} =====\n\n")

print(f" پاراگراف‌های فیلتر شده ذخیره شدند در: {output_path}")

In [ ]:
#8. ترکیب فایل های شماره گذاری ساخته شده

import glob

merged_output_path = os.path.join(Combined_path, "Combined_Sustain_MERGED.txt")

files_to_merge = sorted(
    glob.glob(os.path.join(Combined_path, "Combined_Sustain_text_*.txt")),
    key=lambda x: int(re.search(r"_(\d+)\.txt$", x).group(1))
)

with open(merged_output_path, "w", encoding="utf-8") as outfile:
    for file_path in files_to_merge:
        with open(file_path, "r", encoding="utf-8") as infile:
            content = infile.read()
            outfile.write(content + "\n")

print(f" همه فایل‌ها با موفقیت ترکیب شدند.\n مسیر فایل نهایی: {merged_output_path}")

In [ ]:
# 9. تعداد تکرار کلیدواژه ها در هر فایل

from collections import defaultdict, Counter

file_keyword_counts = {}

sustain_terms_list = list(sustain_terms)

for filename in uploaded_Texts.keys():
    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read().lower()

    counts = {}
    for term in sustain_terms_list:
        pattern = r'\b' + re.escape(term) + r'\b'
        matches = re.findall(pattern, content)
        if matches:
            counts[term] = len(matches)

    file_keyword_counts[filename] = counts

for fname, count_dict in file_keyword_counts.items():
    print(f"\n File: {fname}")
    for kw, cnt in sorted(count_dict.items(), key=lambda x: -x[1]):
        print(f"   {kw} : {cnt}")

In [ ]:
#گزارش تجمیعی از تعداد تکرار کلیدواژه ها در فایل های متنی

import pandas as pd
import os

report_path = "/content/drive/MyDrive/sustainability_table/final_keyword_report.xlsx"

records = []
for fname, count_dict in file_keyword_counts.items():
    for kw, cnt in count_dict.items():
        records.append({"filename": fname, "keyword": kw, "count": cnt})

new_df = pd.DataFrame(records)

#ترکیب گزارشات در هر بار اپلود فایل
if os.path.exists(report_path):
    old_df = pd.read_excel(report_path)
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
else:
    combined_df = new_df

pivot_df = combined_df.pivot_table(
    index="filename",
    columns="keyword",
    values="count",
    aggfunc="sum",
    fill_value=0
)

pivot_df = pivot_df[sorted(pivot_df.columns)]

pivot_df.to_excel(report_path)

print(f" گزارش Pivot ذخیره شد در:\n{report_path}")